## Snapshot and GraphQL

In [1]:
import requests
import pandas as pd
import numpy as np

Snapshot Endpoints

https://docs.snapshot.org/

In [2]:
ENDPOINT_DEMO = 'https://testnet.snapshot.org/graphql'

ENDPOINT_PRO = 'https://hub.snapshot.org/graphql'


ENDPOINT = ENDPOINT_PRO


We can make GraphQL queries with standard POST requests.

In [3]:
# https://datagy.io/python-requests-authentication/

def api(query, params=None):

    response = requests.post(ENDPOINT,
                            headers={                      
                                'accept': 'application/json'
                            },
                            params={
                                'query': query
                            })

    print(response)
    return response.json()['data']


In [4]:
query = """
query {
  space(id: "yam.eth") {
    id
    name
    about
    network
    symbol
    members
  }
}
"""

res = api(query)

<Response [200]>


In [5]:
res


{'space': {'id': 'yam.eth',
  'name': 'Yam',
  'about': 'Only delegated YAM may be used to vote on proposals. You can delegate to yourself or another address here: yam.finance/#/delegate',
  'network': '1',
  'symbol': 'YAM',
  'members': ['0x683A78bA1f6b25E29fbBC9Cd1BFA29A51520De84',
   '0xC3edCBe0F93a6258c3933e86fFaA3bcF12F8D695',
   '0xEC3281124d4c2FCA8A88e3076C1E7749CfEcb7F2']}}

In [6]:
query = """
query {
  spaces(
    first: 20,
    skip: 0,
    orderBy: "created",
    orderDirection: asc
  ) {
    id
    name
    about
    network
    symbol
    strategies {
      name
      params
    }
    admins
    members
    filters {
      minScore
      onlyMembers
    }
    plugins
  }
}
"""

res = api(query)

<Response [200]>


In [ ]:

res

What if want to fetch all spaces? It's a bit cumbersome with standard POST requests, but we can use the [gql](https://pypi.org/project/gql/) client. 

#### GQL Client

In [8]:
# !pip install gql
# !pip install aiohttp 
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport


In [9]:
transport = AIOHTTPTransport(url=ENDPOINT)
client = Client(
    transport=transport
    #fetch_schema_from_transport=True
)


Let's test a simple query.
First, we need build it, when we execute it.

_Note:_ gql will throw an error if the query is malformed.

In [10]:
space_query = gql(
"""
query {
space(id: "yam.eth") {
    id
    name
    about
    network
    symbol
    members
    }
}
""")

In [ ]:
## Error in Jupyter-like environments.
result = client.execute(space_query)
print(result)

In [12]:
result = await client.execute_async(space_query)
print(result)

{'space': {'id': 'yam.eth', 'name': 'Yam', 'about': 'Only delegated YAM may be used to vote on proposals. You can delegate to yourself or another address here: yam.finance/#/delegate', 'network': '1', 'symbol': 'YAM', 'members': ['0x683A78bA1f6b25E29fbBC9Cd1BFA29A51520De84', '0xC3edCBe0F93a6258c3933e86fFaA3bcF12F8D695', '0xEC3281124d4c2FCA8A88e3076C1E7749CfEcb7F2']}}


#### GraphQL Variables

Now we want to fetch _all_ spaces.

We need to use a special feature of GraphQL, that is the ability to pass **variables** into queries.

Variables are identified by the dollar sign `$` and must indicate their type (e.g., `Int`). 

If a variable is mandatory, it is followed by an exclamation mark.

In [13]:
spaces_query = gql("""
  query ($first: Int!, $skip: Int!) {
    spaces(
      first: $first,
      skip: $skip,
      orderBy: "created",
      orderDirection: asc
    ) 
    {
      id
      name
      about
      network
      symbol
      strategies {
        name
        params
      }
      admins
      members
      filters {
        minScore
        onlyMembers
      }
      plugins
    }
  }
""")


In [14]:
## spaces_tmp is a temp variable we can use to store partial computations
## in case an error (e.g., a timeout) occurs
spaces_tmp = []

In [15]:
## Fetch'em all!
spaces = spaces_tmp
first = 1000
skip = len(spaces)
fetch = True
while fetch:
    vars = {"first": first, "skip": skip}
    res = await client.execute_async(spaces_query, variable_values=vars)
    # print(type(res))
    # print(res)
    
    if not res['spaces']:
        print('**I am done fetching!**')
        fetch = False
    else:
        spaces.extend(res['spaces'])
        print(len(spaces))
        skip += first# fetch = False


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000


TransportQueryError: {'message': 'The `skip` argument must not be greater than 30000', 'locations': [{'line': 2, 'column': 3}], 'path': ['spaces']}

In [17]:
len(spaces)


31000

Let's put everything in a Pandas data frame.

In [21]:
df = pd.DataFrame(spaces)

df['id'] = df['id'].astype('string')
df['name'] = df['name'].astype('string')
df['about'] = df['about'].astype('string')
# df['logo'] = df['logo'].astype('string')

In [25]:
df.to_json("../data/daos_snapshot.json", orient="records")

## Visually explore the saved file.
# Is it a mess? Prettify it with VS Code auto-formatter. 
# Don't you remember the shorcut? Check our python_warmup lecture.
 

In [ ]:
df.head()


Now let's create a function to make our life easier.

The function `gql_all` will fetch all paginated responses from an endpoint until it returns an empty response.

_Notice:_ the _async_ keyword and the _try/except_ statement

In [24]:
async def gql_all(query, field, first=1000, skip=0, initial_list=None, 
                  counter = True):

    out = []
    
    if initial_list:
        out = initial_list
        skip = len(out)

    fetch = True
    try:
        while fetch:
            vars = {"first": first, "skip": skip}
            res = await client.execute_async(query, variable_values=vars)
            # print(type(res))
            # print(res)
            
            if not res[field]:
                print('**I am done fetching!**')
                fetch = False
            else:
                out.extend(res[field])
                skip += first# fetch = False
                if counter: print(len(out))
    except:
        print("**An error occurred, exiting early.**")
    
    return out

In [25]:
res = await gql_all(spaces_query, "spaces")

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
**An error occurred, exiting early.**


In [27]:
len(res)

31000